In [49]:
from numpy import array, round
from math import log, sqrt
from random import random
from scipy.stats import rankdata
import sys
from gensim.models import Word2Vec

In [9]:
# Read nvda.group data
import csv
import pandas as pd

f = open('C:\\mallet-2.0.8\\CSE519\\clean_data.txt', 'r')

with f as ins:
    array = []
    for line in ins:
        array.append(line)
        
temp_data = array
data_token = []

for d in temp_data:
    data_token.append(d.split())

flat_list = [item for sublist in data_token for item in sublist]

from collections import Counter

word_freq = Counter(flat_list)
word_freq_log = {}

for k in word_freq.keys():
    word_freq_log[k] = log(word_freq[k])

In [57]:
def rgbColorMix(fromColor, toColor, resolution, randomness = False):
    #fromColor, toColor rgb (255 max) tuple
    #resolution, how many truple to return inbetween
    #(starts at from, but comes up one short of ToColor)
    #print(fromColor)
    #print(toColor)
    (fromColor, toColor) = (array(fromColor), array(toColor))
    fromTo = toColor - fromColor #how to get from fromColor to toColor
    fromToInc = fromTo / float(resolution)
    gradientColors = []
    for i in range(resolution):
        gradientColors.append(tuple([int(x) for x in round(fromColor + (i * fromToInc))]))
    if randomness: 
        for i in range(len(gradientColors)): 
            color = gradientColors[i]
            newcolor = []
            for value in color:
                value += 20 - randint(0, 40)
                value = max(0, min(255, value))
                newcolor.append(value)
            gradientColors[i] = tuple(newcolor)

    #print gradientColors[0:4], gradientColors[-4:] #debug
    return gradientColors

In [4]:
def freqToColor(freq, maxFreq = 1000, resolution=64, colorScheme='multi'):

	#print freq, maxFreq

	perc = freq / float(maxFreq)
	(red, green, blue) = (0, 0, 0)
	if colorScheme=='multi':
	#print "%d %d %.4f" %(freq, maxFreq, perc)#debug
		if perc < 0.17: #grey to darker grey
			(red, green, blue) = rgbColorMix((168, 168, 168),(124, 124, 148), resolution)[int(((1.00-(1-perc))/0.17)*resolution) - 1]
		elif perc >= 0.17 and perc < 0.52: #grey to blue
			(red, green, blue) = rgbColorMix((124, 124, 148), (32, 32, 210), resolution)[int(((0.830-(1-perc))/0.35)*resolution) - 1]
		elif perc >= 0.52 and perc < 0.90: #blue to red
			(red, green, blue) = rgbColorMix((32, 32, 210), (200, 16, 32), resolution)[int(((0.48-(1-perc))/0.38)*resolution) - 1]
		else: #red to dark red
			(red, green, blue) = rgbColorMix((200, 16, 32), (128, 0, 0), resolution)[int(((0.10-(1-perc))/0.10)*resolution) - 1]
	# blue:
	elif colorScheme=='blue':
		if perc <= 0.50: #light blue to med. blue
			(red, green, blue) = rgbColorMix((170, 170, 210), (90, 90, 240), resolution)[int(((1.00-(1-perc))/0.5)*resolution) - 1]
		else: #med. blue to strong blue
			(red, green, blue) = rgbColorMix((90, 90, 240), (30, 30, 140), resolution)[int(((0.5-(1-perc))/0.5)*resolution) - 1]
				# blue:
	elif colorScheme=='old_blue':
		if perc < 0.50: #light blue to med. blue
			(red, green, blue) = rgbColorMix((76, 76, 236), (48, 48, 156), resolution)[int(((1.00-(1-perc))/0.5)*resolution) - 1]
		else: #med. blue to strong blue
			(red, green, blue) = rgbColorMix((48, 48, 156), (0, 0, 110), resolution)[int(((0.5-(1-perc))/0.5)*resolution) - 1]
	#red:
	elif colorScheme=='red': 
		if perc < 0.50: #light red to med. red
			(red, green, blue) = rgbColorMix((236, 76, 76), (156, 48, 48), resolution)[int(((1.00-(1-perc))/0.5)*resolution) - 1]
		else: #med. red to strong red
			(red, green, blue) = rgbColorMix((156, 48, 48), (110, 0, 0), resolution)[int(((0.5-(1-perc))/0.5)*resolution) - 1]
	elif colorScheme=='green': 
		(red, green, blue) = rgbColorMix((166, 247, 178), (27, 122, 26), resolution)[int((1.00-(1-perc))*resolution) - 1]

	elif colorScheme == 'test':
		(red, green, blue) = (255, 255, 255)
	#red+randomness:
	elif colorScheme=='red-random':
		if perc < 0.50: #light blue to med. blue
			(red, green, blue) = rgbColorMix((236, 76, 76), (156, 48, 48), resolution, True)[int(((1.00-(1-perc))/0.5)*resolution) - 1]
		else: #med. blue to strong blue
			(red, green, blue) = rgbColorMix((156, 48, 48), (110, 0, 0), resolution, True)[int(((0.5-(1-perc))/0.5)*resolution) - 1]


	#print "(%d %d %d)" %(red, green, blue)#debug

	htmlcode = "%02s%02s%02s" % (hex(red)[2:], hex(green)[2:], hex(blue)[2:])
	return htmlcode.replace(' ', '0')

In [59]:
from xml.dom import minidom
DIR = 'C:\\mallet-2.0.8\\CSE519\\'

def generate_topic_color(topic_no, limit):
    TOPIC_NO = topic_no

    # parse an xml file by name
    mydoc = minidom.parse(DIR + 'clean_data.xml')

    items = mydoc.getElementsByTagName('topic')

    f = open(DIR + "topic_"+str(TOPIC_NO)+".txt", "w")

    item = items[TOPIC_NO].getElementsByTagName('word')
    
    freq = 0
    
    for elem in item[:limit]: 
        fr = float(elem.attributes['prob'].value)*1000
        if fr>freq:
            freq=fr
    
    for elem in item[:limit]:  
        
        #if word_freq_log[elem.firstChild.data]>freq:
        if elem.firstChild.data not in word_freq: 
            continue
#         print(elem.firstChild.data)
#         print(elem.attributes['prob'].value)
#         print(freq)
        clr = freqToColor(float(elem.attributes['prob'].value)*1000, freq)
        size = word_freq[elem.firstChild.data]
        
        #size = word_freq[elem.firstChild.data]
        #clr = freqToColor(float(elem.attributes['prob'].value)*1000, freq)
        
        #print(elem.firstChild.data+ "\t"+ str(size) + "\t"+ str(clr))
        f.write(elem.firstChild.data+ "\t"+ str(size) + "\t"+ str(clr)+"\n")
    f.close()

In [60]:
for x in range(0,10):
    generate_topic_color(x,80)